# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [41]:
def build_classifiers(First, Second, Third):
    cls_list = []
    trained_classifiers = []
    
    first_classifier = First
    cls_list.append(first_classifier)
    second_classifier = Second
    cls_list.append(second_classifier)
    third_classifier = Third
    cls_list.append(third_classifier)

    for classifier in cls_list:
        classifier.fit(data_set, labels)
        trained_classifiers.append(classifier)
    return trained_classifiers

In [44]:
def build_stacked_classifier(classifiers, stack_classifier):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    classifier = stack_classifier
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = classifier # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [77]:
import warnings
warnings.filterwarnings('ignore')

In [78]:
from itertools import combinations
classifier_list = [LinearRegression(), KNeighborsClassifier(), SVC(), DecisionTreeClassifier(), GaussianNB(), QuadraticDiscriminantAnalysis()]
meta_list = [KNeighborsClassifier(), SVC(), DecisionTreeClassifier(), GaussianNB(), QuadraticDiscriminantAnalysis()]
combo_list=[]
results = []

for combo in combinations(classifier_list, 3):
    combo_list.append(combo)
for meta in meta_list:
    for combo in combo_list:
        if ((combo[0] == meta) or (combo[1] == meta) or (combo[2] == meta)):
            continue
        else:
            classifiers = build_classifiers(combo[0], combo[1], combo[2])
            predicted = build_stacked_classifier(classifiers, meta)
            accuracy = accuracy_score(test_labels, predicted)
            results.append(list([accuracy, combo[0], combo[1], combo[2], meta]))

sorted_results = sorted(results, key=lambda x: x[0], reverse=True)
for result in sorted_results:
    print(result)


[1.0, LinearRegression(), KNeighborsClassifier(), SVC(), GaussianNB()]
[1.0, LinearRegression(), KNeighborsClassifier(), DecisionTreeClassifier(), GaussianNB()]
[1.0, LinearRegression(), KNeighborsClassifier(), GaussianNB(), GaussianNB()]
[1.0, LinearRegression(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis(), GaussianNB()]
[1.0, LinearRegression(), SVC(), DecisionTreeClassifier(), GaussianNB()]
[1.0, LinearRegression(), SVC(), GaussianNB(), GaussianNB()]
[1.0, LinearRegression(), SVC(), QuadraticDiscriminantAnalysis(), GaussianNB()]
[1.0, LinearRegression(), DecisionTreeClassifier(), GaussianNB(), GaussianNB()]
[1.0, LinearRegression(), DecisionTreeClassifier(), QuadraticDiscriminantAnalysis(), GaussianNB()]
[1.0, LinearRegression(), GaussianNB(), QuadraticDiscriminantAnalysis(), GaussianNB()]
[1.0, KNeighborsClassifier(), SVC(), DecisionTreeClassifier(), GaussianNB()]
[1.0, KNeighborsClassifier(), SVC(), GaussianNB(), GaussianNB()]
[1.0, KNeighborsClassifier(), SVC(), Quadr

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [79]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [92]:
def set_new_weights(model):
    predicted = model.predict(test_set)
    I = (predicted != test_labels).astype(int)
    return (1 + I) / np.sum(1 + I)

Train the classifier with the code below:

In [93]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00131666 0.00065833 0.00131666 0.00065833 0.00131666 0.00065833
 0.00131666 0.00065833 0.00065833 0.00065833 0.00065833 0.00065833
 0.00065833 0.00065833 0.00065833 0.00065833 0.00131666 0.00065833
 0.00131666 0.00065833 0.00065833 0.00131666 0.00131666 0.00065833
 0.00131666 0.00131666 0.00131666 0.00065833 0.00131666 0.00065833
 0.00065833 0.00065833 0.00065833 0.00131666 0.00065833 0.00131666
 0.00065833 0.00065833 0.00131666 0.00065833 0.00065833 0.00065833
 0.00065833 0.00131666 0.00065833 0.00131666 0.00131666 0.00131666
 0.00131666 0.00065833 0.00131666 0.00131666 0.00065833 0.00131666
 0.00131666 0.00065833 0.00065833 0.00065833 0.00065833 0.00131666
 0.00131666 0.00131666 0.00131666 0.00131666 0.00065833 0.00131666
 0.00065833 0.00131666 0.00131666 0.00131666 0.00131666 0.00131666
 0.00065833 0.00065833 0.00131666 0.00065833 0.00131666 0.00065833
 0.00131666 0.00131666 0.00065833 0.00065833 0.00131666 0.00065833
 0.00065833 0.00131666 0.00065833 0.00131666 0.00065833 0.0013

Set the validation data set:

In [94]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [117]:
def get_prediction(x):
    w_output = []
    for i, classifier in enumerate(classifiers):
        pred = 
        w_output.append()
    return np.where(np.sum(w_output, axis=0) / len(classifiers) >= 0.5, 1, 0)

Test it:

In [118]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
